# Projet : “Taxi Fleet Optimization – Big Data Analysis” 
### Objectif : 
Analyser des millions de courses taxi pour optimiser la flotte, prédire la demande, détecter les pics. 
Dataset Kaggle NYC Taxi Trip Dataset : "https://www.kaggle.com/datasets/kentonnlp/nyc-taxi-trip-duration "
### Idées Spark : 
- Heatmap des zones les plus demandées 
- Détection des heures de rush 
- Prédiction du temps de trajet (MLlib Regression) 
- Clustering des zones (KMeans)

***Étape 0 : Préparer d'environnement Spark***


In [1]:
//Imports
/*
  Ici nous importons tous les packages nécessaires pour Spark SQL et DataFrame.
  - SparkSession: point d'entrée pour Spark
  - functions: fonctions SQL utiles (col, lit, round, udf, avg, count, etc.)
  - types: pour les types de colonnes si besoin
  - Window: pour les fonctions de fenêtrage (rolling, rank, etc.)
*/
import org.apache.spark.sql.{SparkSession, DataFrame, Row}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql.expressions.Window

In [2]:
val spark = SparkSession.builder()
  .appName("NYC Taxi Deep EDA")
  .config("spark.master", "local[*]")
  .getOrCreate()

import spark.implicits._
spark.sparkContext.setLogLevel("WARN") // Réduction du niveau de logs pour ne voir que WARN ou plus


spark = org.apache.spark.sql.SparkSession@74d9532c


org.apache.spark.sql.SparkSession@74d9532c

***Interprétation :*** Cette cellule prépare l'environnement Spark et importe tout ce qui est nécessaire pour manipuler les données. Le local[*] permet d'utiliser tous les cœurs CPU disponibles.

***Étape 1 : Ingestion des données***

In [3]:
//CONFIG
val inputCsv = "data/NYC.csv"
val outputsDir = "outputs"


inputCsv = data/NYC.csv
outputsDir = outputs


outputs

***Interprétation :*** Tous les outputs (analyses, heatmaps, pivot tables) seront écrits dans ce dossier.

In [4]:
//Charger le dataset CSV
println("==> Loading CSV...")

val dfRaw = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .csv(inputCsv)

dfRaw.printSchema() //inferSchema permet de détecter automatiquement le type des colonnes
dfRaw.show(5, truncate = false)

println(s"Total rows (raw): ${dfRaw.count()}") // Nombre total de lignes

// Vérification rapide des valeurs distinctes sur les premières colonnes
dfRaw.columns.take(8).foreach { c =>
  val d = dfRaw.select(col(c)).distinct().count()
  println(s"  $c -> $d distinct")
}


==> Loading CSV...
root
 |-- id: string (nullable = true)
 |-- vendor_id: integer (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- trip_duration: integer (nullable = true)

+---------+---------+-------------------+-------------------+---------------+------------------+------------------+------------------+------------------+------------------+-------------+
|id       |vendor_id|pickup_datetime    |dropoff_datetime   |passenger_count|pickup_longitude  |pickup_latitude   |dropoff_longitude |dropoff_latitude  |store_and_fwd_flag|trip_duration|
+---------+---------+-------------------+-------------------+---------------+-----------------

dfRaw = [id: string, vendor_id: int ... 9 more fields]


[id: string, vendor_id: int ... 9 more fields]

***Interprétation :****

- Les types ont été correctement inférés par Spark (ex: vendor_id est un int, les coordonnées sont des double).

- Les dates sont encore en string : il faudra les convertir en Timestamp pour des analyses temporelles.

- Les colonnes id et store_and_fwd_flag sont des chaînes (string).

- Il y a 1,458,644 trajets dans le dataset : un dataset très volumineux, parfait pour Spark et l’analyse Big Data.

***Aperçu des données (5 premières lignes)***

Quelques observations :

- id semble unique pour chaque course.

- vendor_id a des valeurs 1 ou 2 (il s’agit probablement des compagnies de taxi).

- pickup_datetime et dropoff_datetime sont sous format YYYY-MM-DD HH:MM:SS.

- Les coordonnées (pickup_longitude/latitude, dropoff_longitude/latitude) sont réalistes pour NYC (longitude ~ -73 / latitude ~ 40).

- trip_duration est en secondes, et varie beaucoup (ex: 429 sec ≈ 7 min, 2124 sec ≈ 35 min).

***Valeurs distinctes par colonne :***
| Colonne             | Distincts  | Interprétation                                                                                            |
| ------------------- | ---------- | --------------------------------------------------------------------------------------------------------- |
| `id`                | 1,458,644  | Chaque course a un identifiant unique, pas de doublons.                                                   |
| `vendor_id`         | 2          | Deux fournisseurs/compagnies de taxi seulement.                                                           |
| `pickup_datetime`   | 1,380,222  | La majorité des dates sont uniques, mais certaines heures/dates se répètent (ex: trajets au même moment). |
| `dropoff_datetime`  | 1,380,377  | Très proche de `pickup_datetime`, donc la plupart des courses ont des durées uniques.                     |
| `passenger_count`   | 10         | Seules 10 valeurs différentes : prob. 1 à 6 passagers + valeurs aberrantes.                               |
| `pickup_longitude`  | 23,047     | Beaucoup de coordonnées uniques : la majorité des trajets commencent à des endroits différents.           |
| `pickup_latitude`   | 45,245     | Idem longitude. La différence longitude/latitude montre une forte densité spatiale (ex: Manhattan).       |
| `dropoff_longitude` | 33,821     | Plus de variation que pickup_longitude → certains trajets finissent à des endroits plus variés.           |
| `dropoff_latitude`  | non listée | Probablement similaire à dropoff_longitude.                                                               |


***Observations importantes :***

Les trajets sont très dispersés géographiquement, surtout pour les dropoffs.

passenger_count est limité à 10 valeurs distinctes, donc on peut faire une analyse de distribution simple.

La majorité des identifiants (id) sont uniques, donc aucune duplication immédiate.

***Étape 2 : Prétraitement et Features Engineering***

In [5]:
// Nettoyage basique des données
/*
  1) Suppression des lignes contenant des valeurs nulles
  2) Filtrage des trip_duration improbables (<1 min ou >27h)
  3) Filtrage des passagers invalides (0 ou >6)
*/

//Suppression des lignes avec valeurs nulles
val dfNonNull = dfRaw.na.drop()
/*Toute ligne contenant au moins une valeur manquante est supprimée.
Cela garantit que toutes les colonnes seront non nulles pour les analyses suivantes.*/

val dfFiltered = dfNonNull
  .filter($"trip_duration".isNotNull && $"trip_duration" > 60 && $"trip_duration" < 100000)//Filtrage sur trip_duration
/*Les trajets de moins de 1 minute (60 secondes) sont considérés comme des erreurs ou des trajets très courts improbables.
Les trajets de plus de 100 000 secondes (~27h) sont également supprimés, car ce sont probablement des anomalies ou des erreurs de saisie.*/

  .filter($"passenger_count".isNotNull && $"passenger_count" > 0 && $"passenger_count" < 7) //Filtrage sur passenger_count
/*Les trajets avec 0 passager sont invalides.

Les trajets avec plus de 6 passagers sont rares pour un taxi standard, donc considérés comme aberrants.*/
println(s"Nombre de lignes après nettoyage: ${dfFiltered.count()}")


Nombre de lignes après nettoyage: 1449843


dfNonNull = [id: string, vendor_id: int ... 9 more fields]
dfFiltered = [id: string, vendor_id: int ... 9 more fields]


[id: string, vendor_id: int ... 9 more fields]

***Interprétation :***

- Le dataset initial avait 1,458,644 lignes.

- Après suppression des valeurs nulles et filtrages, il reste 1,449,843 lignes.

- Donc environ 8,801 lignes ont été supprimées (~0,6 % du dataset).

>Cela signifie que la grande majorité des données est valide et exploitable.

***Points clés sur la qualité des données***

- Les valeurs nulles sont rares → le dataset est assez propre.

- Les anomalies sur trip_duration ou passenger_count sont très limitées.

- Après nettoyage, le dataset est prêt pour :

    - Analyse descriptive (durée moyenne, distribution des passagers, fréquence des trajets par heure/jour).

    - Analyse spatiale (coordonnées de pickup/dropoff).

In [6]:
// Parsing timestamps et features temporelles
/*
  Conversion des colonnes pickup_datetime et dropoff_datetime en type Timestamp
  - Création de nouvelles colonnes temporelles utiles pour l'analyse:
    - pickup_hour : heure de la course
    - pickup_dayofweek : jour de la semaine
    - pickup_month : mois
    - pickup_weekday : indicateur 1=semaine, 0=weekend
*/

val dfTime = dfFiltered
    //convertir les colonnes pickup_datetime et dropoff_datetime en Timestamp
  .withColumn("pickup_ts", to_timestamp($"pickup_datetime","dd/MM/yyyy HH:mm"))
  .withColumn("dropoff_ts", to_timestamp($"dropoff_datetime","dd/MM/yyyy HH:mm"))
    //création des features temporelles utiles pour l’analyse :
  .withColumn("pickup_hour", hour($"pickup_ts")) //pickup_hour : heure du jour (0–23)
  .withColumn("pickup_dayofweek", date_format($"pickup_ts", "E"))  // pickup_dayofweek : jour de la semaine (Mon, Tue…)
  .withColumn("pickup_month", month($"pickup_ts")) //pickup_month : mois (1–12)
  //pickup_weekday : 1 si jour de semaine, 0 si weekend
  .withColumn("pickup_weekday",
      when(dayofweek($"pickup_ts").between(2,6), 1).otherwise(0)
  )


// Vérification
dfTime.select("pickup_ts","dropoff_ts","pickup_hour","pickup_dayofweek","pickup_month","pickup_weekday").show(5)


+---------+----------+-----------+----------------+------------+--------------+
|pickup_ts|dropoff_ts|pickup_hour|pickup_dayofweek|pickup_month|pickup_weekday|
+---------+----------+-----------+----------------+------------+--------------+
|     null|      null|       null|            null|        null|             0|
|     null|      null|       null|            null|        null|             0|
|     null|      null|       null|            null|        null|             0|
|     null|      null|       null|            null|        null|             0|
|     null|      null|       null|            null|        null|             0|
+---------+----------+-----------+----------------+------------+--------------+
only showing top 5 rows



dfTime = [id: string, vendor_id: int ... 15 more fields]


[id: string, vendor_id: int ... 15 more fields]

***Observation :*** toutes les valeurs sont null, sauf pickup_weekday qui par défaut prend 0 (weekend).

***Pourquoi toutes les dates sont nulles***

La cause est le format de date fourni à to_timestamp :

to_timestamp($"pickup_datetime","dd/MM/yyyy HH:mm")


- La dataset montre des dates comme : "2016-03-14 17:24:55"

- Le format "dd/MM/yyyy HH:mm" ne correspond pas :

    - Les dates ont le format yyyy-MM-dd HH:mm:ss

    - Le format actuel attend jour/mois/année heure:minute et ignore les secondes, donc Spark ne peut pas parser → retourne null.

***Comment corriger***

In [7]:
val dfTime = dfFiltered
  .withColumn("pickup_ts", to_timestamp($"pickup_datetime", "yyyy-MM-dd HH:mm:ss"))
  .withColumn("dropoff_ts", to_timestamp($"dropoff_datetime", "yyyy-MM-dd HH:mm:ss"))
  .withColumn("pickup_hour", hour($"pickup_ts"))
  .withColumn("pickup_dayofweek", date_format($"pickup_ts", "E"))
  .withColumn("pickup_month", month($"pickup_ts"))
  .withColumn("pickup_weekday", when(dayofweek($"pickup_ts").between(2,6), 1).otherwise(0))
// Vérification
dfTime.select("pickup_ts","dropoff_ts","pickup_hour","pickup_dayofweek","pickup_month","pickup_weekday").show(5)


+-------------------+-------------------+-----------+----------------+------------+--------------+
|          pickup_ts|         dropoff_ts|pickup_hour|pickup_dayofweek|pickup_month|pickup_weekday|
+-------------------+-------------------+-----------+----------------+------------+--------------+
|2016-03-14 17:24:55|2016-03-14 17:32:30|         17|             Mon|           3|             1|
|2016-06-12 00:43:35|2016-06-12 00:54:38|          0|             Sun|           6|             0|
|2016-01-19 11:35:24|2016-01-19 12:10:48|         11|             Tue|           1|             1|
|2016-04-06 19:32:31|2016-04-06 19:39:40|         19|             Wed|           4|             1|
|2016-03-26 13:30:55|2016-03-26 13:38:10|         13|             Sat|           3|             0|
+-------------------+-------------------+-----------+----------------+------------+--------------+
only showing top 5 rows



dfTime = [id: string, vendor_id: int ... 15 more fields]


[id: string, vendor_id: int ... 15 more fields]

- Maintenant Spark pourra parser correctement les dates.

- Les nouvelles colonnes pickup_hour, pickup_dayofweek, pickup_month, pickup_weekday auront des valeurs correctes.

- Les colonnes pickup_ts et dropoff_ts sont maintenant des Timestamps, donc exploitables pour des calculs temporels.

- On peut maintenant calculer des durées, grouper par heure, jour ou mois, et effectuer des analyses temporelles fiables.

***Features temporelles créées***
| Colonne            | Exemple | Interprétation                                                                                 |
| ------------------ | ------- | ---------------------------------------------------------------------------------------------- |
| `pickup_hour`      | 17      | L’heure de départ de la course (0–23). Utile pour analyser la répartition horaire des trajets. |
| `pickup_dayofweek` | Mon     | Jour de la semaine au format abrégé. Permet de voir les patterns hebdomadaires.                |
| `pickup_month`     | 3       | Mois du trajet. Permet d’analyser la saisonnalité.                                             |
| `pickup_weekday`   | 1       | 1 si jour de semaine, 0 si weekend. Utile pour différencier activité travail vs loisirs.       |


In [8]:
// Calcul de la distance avec Haversine
/*
  Création d'une UDF pour calculer la distance "à vol d'oiseau" en km entre
  le point de départ et d'arrivée. La formule Haversine est précise pour la sphère terrestre.
*/

val haversine = udf((lat1: Double, lon1: Double, lat2: Double, lon2: Double) => {
  val R = 6371.0  // rayon de la Terre en km
  val dLat = math.toRadians(lat2 - lat1)
  val dLon = math.toRadians(lon2 - lon1)
  val a = math.pow(math.sin(dLat/2),2) +
          math.cos(math.toRadians(lat1)) *
          math.cos(math.toRadians(lat2)) *
          math.pow(math.sin(dLon/2),2)
  val c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
  R * c
})

// Application de la fonction
val dfDist = dfTime.withColumn("distance_km", haversine(
  $"pickup_latitude", $"pickup_longitude", $"dropoff_latitude", $"dropoff_longitude"
))

dfDist.select("pickup_latitude","pickup_longitude","dropoff_latitude","dropoff_longitude","distance_km").show(5)


+------------------+------------------+------------------+------------------+------------------+
|   pickup_latitude|  pickup_longitude|  dropoff_latitude| dropoff_longitude|       distance_km|
+------------------+------------------+------------------+------------------+------------------+
| 40.76793670654297| -73.9821548461914|40.765602111816406|-73.96463012695312|1.4985207796474773|
|40.738563537597656|-73.98041534423828| 40.73115158081055|-73.99948120117188|1.8055071687958235|
|40.763938903808594| -73.9790267944336|40.710086822509766|-74.00533294677734|  6.38509849525294|
|   40.719970703125|-74.01004028320312| 40.70671844482422|-74.01226806640625| 1.485498422771006|
|40.793209075927734|-73.97305297851562| 40.78252029418945| -73.9729232788086| 1.188588459334221|
+------------------+------------------+------------------+------------------+------------------+
only showing top 5 rows



haversine = SparkUserDefinedFunction($Lambda$4586/0x00000008417c7840@5066e376,DoubleType,List(Some(class[value[0]: double]), Some(class[value[0]: double]), Some(class[value[0]: double]), Some(class[value[0]: double])),None,false,true)
dfDist = [id: string, vendor_id: int ... 16 more fields]


[id: string, vendor_id: int ... 16 more fields]

***Objectif du code***

- Calculer la distance “à vol d’oiseau” entre le point de départ (pickup) et d’arrivée (dropoff) pour chaque course.

- Utilisation de la formule Haversine, qui tient compte de la sphère terrestre (plus précise que la distance euclidienne simple).

- Résultat en kilomètres dans une nouvelle colonne distance_km.

***Aperçu des résultats***
Exemple des 5 premières lignes :
| pickup_lat | pickup_lon | dropoff_lat | dropoff_lon | distance_km |
| ---------- | ---------- | ----------- | ----------- | ----------- |
| 40.768     | -73.982    | 40.766      | -73.965     | 1.50        |
| 40.739     | -73.980    | 40.731      | -73.999     | 1.81        |
| 40.764     | -73.979    | 40.710      | -74.005     | 6.39        |
| 40.720     | -74.010    | 40.707      | -74.012     | 1.49        |
| 40.793     | -73.973    | 40.783      | -73.973     | 1.19        |

***Interprétation :***

- Les distances semblent réalistes pour des trajets à NYC :

    - Courtes courses → ~1–2 km

    - Courses plus longues → 6 km et plus

- On peut maintenant analyser :

    - Distribution des distances

    - Relation entre distance_km et trip_duration (vitesse moyenne)

    - Détection de trajets aberrants (distance très grande ou nulle)

***Utilité de cette feature***

distance_km est essentielle pour :

- Études de performance des trajets (temps vs distance)

- Détection des anomalies

- Visualisations géospatiales et heatmaps

- Éventuelles modélisations prédictives (ex: prédire trip_duration)

In [9]:
// Trip duration en minutes et vitesse
/*
  - trip_min : conversion de la durée en secondes vers minutes
  - speed_kmph : calcul de la vitesse moyenne en km/h
  Cette feature est utile pour détecter les courses anormales ou trop lentes/rapides.
*/

val dfSpeed = dfDist.withColumn("trip_min", $"trip_duration"/60.0)
  .withColumn("speed_kmph", $"distance_km"/($"trip_min"/60.0))

dfSpeed.select("trip_duration","trip_min","distance_km","speed_kmph").show(5)


dfSpeed = [id: string, vendor_id: int ... 18 more fields]


+-------------+-----------------+------------------+------------------+
|trip_duration|         trip_min|       distance_km|        speed_kmph|
+-------------+-----------------+------------------+------------------+
|          455|7.583333333333333|1.4985207796474773| 11.85642814666136|
|          663|            11.05|1.8055071687958235| 9.803658835090443|
|         2124|             35.4|  6.38509849525294|10.822200839411764|
|          429|             7.15| 1.485498422771006|12.465721030246204|
|          435|             7.25| 1.188588459334221| 9.836594146214244|
+-------------+-----------------+------------------+------------------+
only showing top 5 rows



[id: string, vendor_id: int ... 18 more fields]

***Objectif du code:***

- trip_min : conversion de la durée du trajet de secondes → minutes, plus lisible pour interprétation.

- speed_kmph : calcul de la vitesse moyenne du trajet en km/h.

- Utile pour détecter des trajets anormalement rapides ou lents, ou des erreurs de saisie dans les données.

***Interprétation :***

- La vitesse moyenne des trajets est raisonnable pour NYC, autour de 10–12 km/h, ce qui correspond à la circulation urbaine.

- Même pour des trajets plus longs (35 min / 6.39 km), la vitesse moyenne reste cohérente (~10.8 km/h).

- Les valeurs permettent de détecter les anomalies :

    - Vitesse trop faible (<5 km/h) → peut indiquer arrêt prolongé ou erreur dans les données.

    - Vitesse trop élevée (>100 km/h) → probablement une erreur de saisie (ex: coordonnée ou durée).

***Utilité de ces nouvelles features***

- trip_min : plus intuitive que trip_duration pour la lecture et les analyses statistiques.

- speed_kmph :

    - Permet de filtrer les courses anormales.

    - Peut être utilisée comme feature pour un modèle prédictif de durée.

    - Utile pour analyses urbaines (identifier zones de trafic lent/rapide).

In [10]:
// Agrégations temporelles
/*
  Analyse du nombre de courses par heure et par jour de la semaine
  - tripsByHour : utile pour détecter les heures de pointe
  - tripsByWeekday : utile pour détecter les jours les plus chargés
*/

val tripsByHour = dfSpeed.groupBy("pickup_hour").count().orderBy("pickup_hour")
tripsByHour.show(24)

val tripsByWeekday = dfSpeed.groupBy("pickup_dayofweek").count().orderBy(desc("count"))
tripsByWeekday.show()


+-----------+-----+
|pickup_hour|count|
+-----------+-----+
|          0|52840|
|          1|38290|
|          2|27714|
|          3|20684|
|          4|15558|
|          5|14789|
|          6|32962|
|          7|55268|
|          8|66732|
|          9|67320|
|         10|65100|
|         11|68103|
|         12|71466|
|         13|71096|
|         14|73831|
|         15|71366|
|         16|63863|
|         17|76015|
|         18|90152|
|         19|89858|
|         20|83653|
|         21|83721|
|         22|80073|
|         23|69389|
+-----------+-----+

+----------------+------+
|pickup_dayofweek| count|
+----------------+------+
|             Fri|222192|
|             Sat|219567|
|             Thu|217232|
|             Wed|209004|
|             Tue|201587|
|             Sun|194005|
|             Mon|186256|
+----------------+------+



tripsByHour = [pickup_hour: int, count: bigint]
tripsByWeekday = [pickup_dayofweek: string, count: bigint]


[pickup_dayofweek: string, count: bigint]

***Courses par heure (tripsByHour)***

- Les trajets sont moins nombreux la nuit (minuit–5h : 15 558 à 52 840 trajets).

- La fréquentation augmente à partir de 6h, avec des pics visibles :

    - 8h–10h → heure de pointe du matin (commuters).

    - 17h–19h → pic l’après-midi/soir (retours du travail).

- Le maximum est à 18h (90 152 trajets) → fort trafic urbain.

***Courses par jour de la semaine (tripsByWeekday)***

- Vendredi et samedi sont les jours les plus chargés : 222 192 et 219 567 trajets → activités sociales et travail.

- Lundi et dimanche sont les moins chargés : 186 256 et 194 005 trajets.

- Le dataset reflète le pattern urbain classique : plus de trajets en fin de semaine et pendant les heures de pointe.


***En résumé :*** Les données montrent une activité élevée aux heures de pointe et une répartition hebdomadaire typique, utile pour analyses de trafic ou planification de services.

In [11]:
// Création de bins pour heatmap
/*
  Pour visualiser les zones les plus demandées, nous arrondissons les coordonnées
  afin de créer des "bins" (zones) de latitude/longitude.
*/

val dfHeat = dfSpeed
  .withColumn("pickup_lat_bin_0_01", round($"pickup_latitude", 2))
  .withColumn("pickup_lon_bin_0_01", round($"pickup_longitude", 2))

val heatmap = dfHeat.groupBy("pickup_lat_bin_0_01","pickup_lon_bin_0_01").count().orderBy(desc("count"))
heatmap.show(20)


dfHeat = [id: string, vendor_id: int ... 20 more fields]
heatmap = [pickup_lat_bin_0_01: double, pickup_lon_bin_0_01: double ... 1 more field]


+-------------------+-------------------+-----+
|pickup_lat_bin_0_01|pickup_lon_bin_0_01|count|
+-------------------+-------------------+-----+
|              40.76|             -73.97|89872|
|              40.75|             -73.99|89744|
|              40.76|             -73.98|77421|
|              40.75|             -73.98|72329|
|              40.74|             -73.99|65562|
|              40.76|             -73.99|65369|
|              40.77|             -73.96|51198|
|              40.73|             -73.99|50993|
|              40.77|             -73.98|47558|
|              40.73|              -74.0|44185|
|              40.74|              -74.0|43723|
|              40.74|             -73.98|41274|
|              40.78|             -73.96|38929|
|              40.78|             -73.95|37617|
|              40.78|             -73.98|35930|
|              40.72|             -73.99|34007|
|              40.75|             -73.97|32117|
|              40.77|             -73.95

[pickup_lat_bin_0_01: double, pickup_lon_bin_0_01: double ... 1 more field]

**Interprétation**:
Les premières lignes montrent des bins autour de :

Latitude ≈ 40.75 – 40.78

Longitude ≈ -73.95 – -74.00

=> Ces coordonnées correspondent à des zones très centrales de Manhattan, notamment :

Midtown Manhattan

Times Square

Grand Central

Zones de bureaux, tourisme et transports

Par exemple :

La zone (40.76, -73.97) compte ~89 000 trajets, ce qui indique une très forte demande.

Plusieurs bins adjacents ont des volumes similaires, ce qui confirme une concentration spatiale importante des pickups.

In [12]:
// Cell 9: Export CSV pour visualisation
/*
  On exporte les résultats d'analyse pour créer des visualisations externes
  (par exemple dans Excel, Tableau, PowerBI ou folium/kepler pour cartes)
*/

tripsByHour.coalesce(1).write.mode("overwrite").option("header","true").csv("outputs/trips_by_hour")
tripsByWeekday.coalesce(1).write.mode("overwrite").option("header","true").csv("outputs/trips_by_weekday")
heatmap.coalesce(1).write.mode("overwrite").option("header","true").csv("outputs/heatmap_bins")


In [13]:
// Cell 10: Pivot zone x heure pour clustering
/*
  Création d'un tableau pivot zone (lat/lon bin) x heure
  - chaque cellule contient le nombre de courses
  - préparation pour clustering ou analyse de la demande par zone/heure
*/

val demandPerZoneHour = dfHeat.groupBy("pickup_lat_bin_0_01","pickup_lon_bin_0_01","pickup_hour")
  .agg(count(lit(1)).alias("n_trips"))

val pivotZone = demandPerZoneHour.groupBy("pickup_lat_bin_0_01","pickup_lon_bin_0_01")
  .pivot("pickup_hour")
  .agg(first("n_trips"))
  .na.fill(0)

pivotZone.show(5)
pivotZone.coalesce(1).write.mode("overwrite").option("header","true").csv("outputs/pivot_zone_hourly")


+-------------------+-------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|pickup_lat_bin_0_01|pickup_lon_bin_0_01|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23|
+-------------------+-------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|              40.65|             -73.96| 14| 10|  6|  4|  8|  5|  7|  8|  2|  2|  5|  0|  1|  1|  0|  2|  1|  3|  2|  0|  3|  4|  5| 10|
|              40.81|             -73.99|  0|  0|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  1|
|              40.59|             -73.95|  0|  0|  0|  0|  2|  0|  1|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|              40.68|             -73.82|  1|  1|  0|  0|  0|  0|  0|  1|  2|  2|  1|  0|  1|  0|  0|  0|  0|  1|  2|  0|  0|  3|  2|  3|
|              40.81|             

demandPerZoneHour = [pickup_lat_bin_0_01: double, pickup_lon_bin_0_01: double ... 2 more fields]
pivotZone = [pickup_lat_bin_0_01: double, pickup_lon_bin_0_01: double ... 24 more fields]


[pickup_lat_bin_0_01: double, pickup_lon_bin_0_01: double ... 24 more fields]

**Interprétation** :

La seconde partie crée un tableau pivot où :

- chaque ligne représente une zone géographique (bin latitude/longitude),

- chaque colonne correspond à une heure de la journée (0 à 23),

- chaque valeur indique le nombre de courses dans cette zone à cette heure.

Les valeurs manquantes sont remplacées par 0, ce qui garantit un tableau complet et exploitable pour des analyses avancées.

=>Lecture des résultats

Les résultats montrent que :

- certaines zones ont une activité concentrée à des heures spécifiques,

- d’autres zones présentent une demande très faible ou occasionnelle.

In [14]:
// Filtrer les courses avec distance nulle ou trop grande et vitesse improbable
val dfClean = dfSpeed
  .filter($"distance_km" > 0.01 && $"distance_km" < 500)  // distance plausible
  .filter($"speed_kmph" > 0.5 && $"speed_kmph" < 160)     // vitesse réaliste

println(s"Nombre de lignes après filtrage des outliers: ${dfClean.count()}")

dfClean = [id: string, vendor_id: int ... 18 more fields]


Nombre de lignes après filtrage des outliers: 1441100


[id: string, vendor_id: int ... 18 more fields]

**Interprétation – Filtrage des outliers**:

Ce code nettoie les données en supprimant les courses non réalistes :

- distances nulles ou excessives,

- vitesses trop faibles ou trop élevées.

Après ce filtrage, 1 441 100 courses sont conservées, ce qui garantit une base de données fiable et cohérente pour l’analyse.

In [15]:
//Statistiques descriptives
// Moyenne, médiane et écart type
dfClean.select(
  mean("trip_duration").alias("mean_trip_s"),
  expr("percentile_approx(trip_duration, 0.5)").alias("median_trip_s"),
  stddev("trip_duration").alias("std_trip_s"),
  mean("distance_km").alias("mean_km"),
  expr("percentile_approx(distance_km, 0.5)").alias("median_km"),
  stddev("distance_km").alias("std_km")
).show(false)

+-----------------+-------------+-----------------+------------------+------------------+-----------------+
|mean_trip_s      |median_trip_s|std_trip_s       |mean_km           |median_km         |std_km           |
+-----------------+-------------+-----------------+------------------+------------------+-----------------+
|849.9836048851572|665          |1030.919862183811|3.4733188891949354|2.1168295052175594|3.960717922716133|
+-----------------+-------------+-----------------+------------------+------------------+-----------------+



**Interprétation – Statistiques descriptives**:

Les statistiques montrent que :

- la durée moyenne d’un trajet est d’environ 850 secondes (~14 min), avec une médiane plus faible (665 s), indiquant une distribution asymétrique avec quelques trajets très longs.

- la distance moyenne est de 3,47 km, alors que la médiane est d’environ 2,12 km, ce qui confirme que la majorité des trajets sont courts.

- les écarts types élevés révèlent une forte variabilité des trajets, typique d’un contexte urbain comme New York.

In [16]:
// Routes fréquentes
val dfRoutes = dfClean
  .withColumn("pickup_pair", concat_ws(",", round($"pickup_latitude",3), round($"pickup_longitude",3)))
  .withColumn("dropoff_pair", concat_ws(",", round($"dropoff_latitude",3), round($"dropoff_longitude",3)))
  .withColumn("route", concat_ws("->", $"pickup_pair", $"dropoff_pair"))

val topRoutes = dfRoutes.groupBy("route").count().orderBy(desc("count"))
topRoutes.show(20)
topRoutes.coalesce(1).write.mode("overwrite").option("header","true").csv("outputs/top_routes")

+--------------------+-----+
|               route|count|
+--------------------+-----+
|40.752,-73.978->4...|  107|
|40.774,-73.873->4...|   92|
|40.762,-73.979->4...|   83|
|40.751,-73.994->4...|   70|
|40.774,-73.873->4...|   66|
|40.741,-73.986->4...|   65|
|40.763,-73.982->4...|   61|
|40.751,-73.994->4...|   60|
|40.749,-73.992->4...|   59|
|40.752,-73.977->4...|   57|
|40.77,-73.864->40...|   56|
|40.764,-73.981->4...|   56|
|40.774,-73.871->4...|   55|
|40.752,-73.978->4...|   55|
|40.738,-73.988->4...|   55|
|40.737,-73.989->4...|   54|
|40.751,-73.994->4...|   52|
|40.742,-74.001->4...|   50|
|40.769,-73.863->4...|   50|
|40.75,-73.992->40...|   50|
+--------------------+-----+
only showing top 20 rows



dfRoutes = [id: string, vendor_id: int ... 21 more fields]
topRoutes = [route: string, count: bigint]


[route: string, count: bigint]

**Interprétation – Routes fréquentes**:

Ce code identifie les trajets les plus fréquents en regroupant les points de départ et d’arrivée après arrondi des coordonnées GPS à 3 décimales, ce qui permet de définir des zones approximatives plutôt que des points exacts.

Les résultats montrent que :

- certaines routes sont répétées très souvent, avec plus de 100 trajets identiques,

- ces routes correspondent majoritairement à des zones centrales et stratégiques (gares, centres d’affaires, aéroports).

**Insight**

La répétition de ces trajets indique l’existence de corridors de mobilité très utilisés, reflétant des déplacements réguliers entre des zones clés de la ville.

In [17]:
//Export final dataset nettoyé pour ML
val dfForML = dfClean.select(
  $"pickup_ts", $"dropoff_ts", $"pickup_hour", $"pickup_dayofweek", $"pickup_month", $"pickup_weekday",
  $"passenger_count", $"distance_km", $"trip_duration", $"trip_min", $"speed_kmph"
)

dfForML.write.mode("overwrite").parquet("outputs/nyc_taxi_clean.parquet")

dfForML = [pickup_ts: timestamp, dropoff_ts: timestamp ... 9 more fields]


[pickup_ts: timestamp, dropoff_ts: timestamp ... 9 more fields]

**Export du dataset nettoyé pour ML**

Le code sélectionne les colonnes les plus pertinentes pour l’apprentissage machine (pickup_ts, dropoff_ts, pickup_hour, distance_km, trip_duration, etc.) et les exporte au format Parquet.

- Cela permet de créer un dataset propre et optimisé pour les modèles prédictifs (par exemple prédiction de la durée des trajets).

- Parquet est choisi pour sa compression et son efficacité de lecture avec Spark.

In [18]:
// détection d’anomalies simples
// IQR-based anomalies pour distance et durée
val Array(q1Dist, q3Dist) = dfClean.stat.approxQuantile("distance_km", Array(0.25,0.75), 0.01)
val iqrDist = q3Dist - q1Dist
val anomaliesDist = dfClean.filter($"distance_km" < q1Dist - 1.5*iqrDist || $"distance_km" > q3Dist + 1.5*iqrDist)
println(s"Anomalies distance: ${anomaliesDist.count()}")
anomaliesDist.coalesce(1).write.mode("overwrite").option("header","true").csv("outputs/anomalies_distance")

Anomalies distance: 143542


q1Dist = 1.239383570220805
q3Dist = 3.830537219099838
iqrDist = 2.591153648879033
anomaliesDist = [id: string, vendor_id: int ... 18 more fields]


[id: string, vendor_id: int ... 18 more fields]

**Détection d’anomalies simples (IQR)**

- On calcule l’IQR (interquartile range) sur la distance pour détecter les trajets aberrants : trop courts ou trop longs.

- Exemple : environ 143 542 courses sont considérées comme anomalies sur 1 441 100 trajets.

- Ces anomalies peuvent être exclues ou étudiées séparément pour améliorer la qualité du dataset.

In [19]:
//Corrélations et pivot tables pour ML
//Corrélation distance ↔ durée, vitesse ↔ durée.
//Pivot pickup_hour x passenger_count pour features additionnelles
println(s"Corrélation distance_km ↔ trip_duration : ${dfClean.stat.corr("distance_km","trip_duration")}")
println(s"Corrélation speed_kmph ↔ trip_duration : ${dfClean.stat.corr("speed_kmph","trip_duration")}")

val pivotHourPassenger = dfClean.groupBy("pickup_hour").pivot("passenger_count").agg(avg("trip_duration")).orderBy("pickup_hour")
pivotHourPassenger.show(24)
pivotHourPassenger.coalesce(1).write.mode("overwrite").option("header","true").csv("outputs/pivot_hour_passenger")


Corrélation distance_km ↔ trip_duration : 0.5212754973100158
Corrélation speed_kmph ↔ trip_duration : 0.025787056868200084
+-----------+-----------------+------------------+------------------+------------------+------------------+-----------------+
|pickup_hour|                1|                 2|                 3|                 4|                 5|                6|
+-----------+-----------------+------------------+------------------+------------------+------------------+-----------------+
|          0|779.2117226550502| 808.2224009190121|  791.550042408821| 813.2803738317757| 856.6091875214262|765.4053880692752|
|          1|748.7430871139182| 766.0483682008369| 756.9690831556503| 724.2337209302326| 780.6540307101727|818.7862776025237|
|          2|727.1558289396602| 730.6873853211009|  718.659511472983| 731.0361445783133| 697.9954188481676|725.6741176470588|
|          3|724.9454843071668| 726.9565082323703| 734.9909547738694|  938.751708428246| 712.8064798598949|710.4676258992

pivotHourPassenger = [pickup_hour: int, 1: double ... 5 more fields]


[pickup_hour: int, 1: double ... 5 more fields]

**Corrélations et pivot tables**

**1.Corrélations :**

- distance_km ↔ trip_duration = 0.52 → la durée augmente logiquement avec la distance.

- speed_kmph ↔ trip_duration ≈ 0.03 → la vitesse n’influence presque pas directement la durée totale.

**2.Pivot Hour x Passenger Count :**

- Création d’un tableau où chaque ligne = heure de prise en charge, chaque colonne = nombre de passagers, et chaque cellule = durée moyenne du trajet.

- Utile pour créer des features additionnelles dans les modèles ML, par exemple pour capturer l’effet du nombre de passagers sur la durée selon l’heure de la journée.

**Insight général**

- Le dataset final est prêt pour ML, avec nettoyage des outliers, features temporelles, anomalies identifiées, et structuration en pivot tables pour enrichir les modèles.

- Cela permet d’avoir une vision complète des trajets, incluant leur distribution, anomalies et facteurs influençant la durée.

In [20]:
// Clustering : zones de forte demande par heure
import org.apache.spark.ml.clustering.KMeans
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.functions._

// Créer le vecteur de features (colonnes d'heures 0..23)
val hourCols = (0 to 23).map(_.toString).toArray
val assembler = new VectorAssembler()
  .setInputCols(hourCols)
  .setOutputCol("features")

val pivotZoneFeatures = assembler.transform(pivotZone)

// Appliquer KMeans
val kmeans = new KMeans()
  .setK(5)
  .setSeed(42)
  .setFeaturesCol("features")
  .setPredictionCol("cluster")

val kModel = kmeans.fit(pivotZoneFeatures)

// Ajouter la colonne cluster
val dfClustered = kModel.transform(pivotZoneFeatures)

// Supprimer la colonne 'features' avant l'export CSV
val dfToSave = dfClustered.drop("features")

// Export CSV
dfToSave.coalesce(1)
  .write
  .mode("overwrite")
  .option("header", "true")
  .csv("outputs/zone_clusters")

// Vérification
dfToSave.show(10)


+-------------------+-------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-------+
|pickup_lat_bin_0_01|pickup_lon_bin_0_01|  0|  1|  2|  3|  4|  5|  6|  7|  8|  9| 10| 11| 12| 13| 14| 15| 16| 17| 18| 19| 20| 21| 22| 23|cluster|
+-------------------+-------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-------+
|              40.65|             -73.96| 14| 10|  6|  4|  8|  5|  7|  8|  2|  2|  5|  0|  1|  1|  0|  2|  1|  3|  2|  0|  3|  4|  5| 10|      0|
|              40.81|             -73.99|  0|  0|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  1|      0|
|              40.59|             -73.95|  0|  0|  0|  0|  2|  0|  1|  0|  0|  0|  0|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|  0|      0|
|              40.68|             -73.82|  1|  1|  0|  0|  0|  0|  0|  1|  2|  2|  1|  0|  1|  0|  0|  0|  0|  1|  2|  0|  0

hourCols = Array(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23)
assembler = VectorAssembler: uid=vecAssembler_d248185184dd, handleInvalid=error, numInputCols=24
pivotZoneFeatures = [pickup_lat_bin_0_01: double, pickup_lon_bin_0_01: double ... 25 more fields]
kmeans = kmeans_43879e8b6a0a
kModel = KMeansModel: uid=kmeans_43879e8b6a0a, k=5, distanceMeasure=euclidean, numFeatures=24


dfClustered: org.apache....


KMeansModel: uid=kmeans_43879e8b6a0a, k=5, distanceMeasure=euclidean, numFeatures=24

**Interprétation des résultats :**

- Chaque ligne correspond à une zone (latitude/longitude arrondie) avec :

    - Les nombre de courses par heure (0 → 23)

    - Le cluster auquel appartient la zone

- Les clusters permettent d’identifier :

    - Zones très actives toute la journée

    - Zones actives seulement aux heures de pointe

    - Zones peu fréquentées

Cela aide à la planification opérationnelle, comme l’affectation des taxis ou la prédiction de demande par quartier.

In [21]:
//Préparation ML : train/test split, encodage, scaling
//Pour prédire trip_duration :
import org.apache.spark.ml.feature.{StringIndexer, VectorAssembler, StandardScaler}
import org.apache.spark.ml.Pipeline

// Encodage de store_and_fwd_flag
val indexer = new StringIndexer()
  .setInputCol("store_and_fwd_flag")
  .setOutputCol("store_and_fwd_flag_idx")

// Sélection des features numériques
val numericCols = Array("passenger_count","distance_km","pickup_hour","pickup_weekday","pickup_month")

// Assemblage en un vecteur de features
val assemblerML = new VectorAssembler()
  .setInputCols(numericCols ++ Array("store_and_fwd_flag_idx"))
  .setOutputCol("features_raw")

// Scaling
val scaler = new StandardScaler()
  .setInputCol("features_raw")
  .setOutputCol("features")
  .setWithStd(true)
  .setWithMean(true)

// Split train/test
val Array(trainData, testData) = dfClean.randomSplit(Array(0.8,0.2), seed=42)


indexer = strIdx_0606206c8f83
numericCols = Array(passenger_count, distance_km, pickup_hour, pickup_weekday, pickup_month)
assemblerML = VectorAssembler: uid=vecAssembler_c62d2abf344b, handleInvalid=error, numInputCols=6
scaler = stdScal_7a31efb8a683
trainData = [id: string, vendor_id: int ... 18 more fields]
testData = [id: string, vendor_id: int ... 18 more fields]


[id: string, vendor_id: int ... 18 more fields]

Le code décrit la préparation des données pour un modèle de machine learning visant à prédire la durée des courses (trip_duration). Les étapes réalisées sont les suivantes :

**1.Encodage des variables catégorielles :**
La colonne contenant des indicateurs textuels (comme "Y" ou "N") est transformée en valeurs numériques pour que le modèle puisse les utiliser.

**2.Sélection des caractéristiques pertinentes :**
Les colonnes choisies incluent le nombre de passagers, la distance du trajet, l’heure et le jour de la prise en charge, ainsi que le mois. Ce sont les informations jugées utiles pour prédire la durée de course.

**3.Assemblage des features :**
Toutes les caractéristiques numériques et encodées sont combinées en un seul vecteur, ce qui est le format attendu par les algorithmes de machine learning.

**4.Normalisation des données :**
Les valeurs sont standardisées afin que toutes les caractéristiques aient la même échelle. Cela améliore la performance et la stabilité des modèles.

**5.Séparation du dataset :**
Les données sont divisées en deux parties : 80 % pour l’entraînement du modèle et 20 % pour tester sa performance. Cette séparation permet d’évaluer la capacité du modèle à généraliser sur de nouvelles données.

***Pipeline ML Spark : RandomForest / GBT / XGBoost***

In [22]:
// RandomForest
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.Pipeline

val rf = new RandomForestRegressor()
  .setLabelCol("trip_duration")
  .setFeaturesCol("features")
  .setNumTrees(100)

val pipelineRF = new Pipeline().setStages(Array(indexer, assemblerML, scaler, rf))

val modelRF = pipelineRF.fit(trainData)
val predictionsRF = modelRF.transform(testData)

predictionsRF.select("trip_duration","prediction").show(10)

// Évaluation
val evaluator = new RegressionEvaluator()
  .setLabelCol("trip_duration")
  .setPredictionCol("prediction")
  .setMetricName("rmse")

println(s"RMSE RandomForest: ${evaluator.evaluate(predictionsRF)}")


+-------------+-----------------+
|trip_duration|       prediction|
+-------------+-----------------+
|         1134|825.9041195825154|
|          592|560.0089441817593|
|         1677|886.2195063509994|
|          303|612.2211725289917|
|          189|530.1144007151372|
|          233| 554.217722021075|
|          998|826.8834488134926|
|           95|634.3060539801587|
|         1041|889.1332625294153|
|          500|608.1625194360639|
+-------------+-----------------+
only showing top 10 rows

RMSE RandomForest: 895.9559161395179


rf = rfr_2463b3c1c53e
pipelineRF = pipeline_48cababd3e18
modelRF = pipeline_48cababd3e18
predictionsRF = [id: string, vendor_id: int ... 22 more fields]
evaluator = RegressionEvaluator: uid=regEval_ce83b124e4fe, metricName=rmse, throughOrigin=false


RegressionEvaluator: uid=regEval_ce83b124e4fe, metricName=rmse, throughOrigin=false

**Interpretations:**

**1.Comparaison prédictions vs valeurs réelles :**

- Par exemple, une course réelle de 1134 secondes a été prédite à 826 secondes, une autre de 592 secondes à 560 secondes, etc.

- On remarque que pour des courses courtes, la prédiction est relativement proche, tandis que pour des courses très longues, le modèle sous-estime souvent la durée.

**2. RMSE = 895,96 secondes :**

- Le Root Mean Squared Error indique qu’en moyenne, les prédictions du modèle diffèrent des valeurs réelles d’environ 896 secondes (~15 minutes).

- Cela montre que le modèle capture les tendances globales mais qu’il a encore une marge d’erreur notable, surtout pour les courses très longues ou très courtes.

**3.Tendance générale :**

- La Random Forest fonctionne mieux pour les durées “typical” ou moyennes, mais les valeurs extrêmes (courses très courtes ou très longues) sont moins bien prédites.

- Cela peut être dû à la distribution très dispersée des durées et à la présence de valeurs atypiques même après nettoyage.


En résumé : le modèle est utile pour avoir une estimation générale de la durée d’une course, mais il n’est pas extrêmement précis pour toutes les courses, surtout celles aux extrêmes.

In [23]:
// Gradient Boosted Trees (GBT) 
import org.apache.spark.ml.regression.GBTRegressor 
val gbt = new GBTRegressor()
  .setLabelCol("trip_duration")
  .setFeaturesCol("features")
  .setMaxIter(100)
val pipelineGBT = new Pipeline().setStages(Array(indexer, assemblerML, scaler, gbt)) 
val modelGBT = pipelineGBT.fit(trainData) 
val predictionsGBT = modelGBT.transform(testData) 
println(s"RMSE GBT: ${evaluator.evaluate(predictionsGBT)}")

RMSE GBT: 867.5099355083967


gbt = gbtr_90dfecbc237b
pipelineGBT = pipeline_e0920f277056
modelGBT = pipeline_e0920f277056
predictionsGBT = [id: string, vendor_id: int ... 22 more fields]


[id: string, vendor_id: int ... 22 more fields]

**Interpretations :**
**1. RMSE = 867,51 secondes :**

- Le modèle a une erreur quadratique moyenne d’environ 867 secondes (~14,5 minutes).

- Comparé au Random Forest (RMSE ≈ 896 s), le GBT réduit légèrement l’erreur, ce qui indique qu’il est un peu plus précis pour prédire la durée des courses.

**2. Performance globale :**

- Comme pour Random Forest, le modèle capture globalement les tendances des durées de courses.

- Les courses très longues ou très courtes restent difficiles à prédire avec précision, mais le GBT gère mieux certaines valeurs extrêmes grâce à son apprentissage itératif et pondéré.

**3.Tendance :**

- Les prédictions GBT sont généralement plus proches des valeurs réelles, surtout pour les courses dont la durée est proche de la moyenne.

- Les valeurs atypiques continuent de poser un défi, mais GBT est plus robuste aux variations que RF.


En résumé :

- Le GBT est légèrement plus performant que Random Forest pour ce jeu de données.

- Il reste utile pour une estimation générale des durées de courses, mais des erreurs importantes peuvent subsister sur les courses extrêmes.

In [24]:
val evaluator = new RegressionEvaluator()
  .setLabelCol("trip_duration")
  .setPredictionCol("prediction")
  .setMetricName("rmse") // Root Mean Squared Error

val rmseRF = evaluator.evaluate(predictionsRF)
val rmseGBT = evaluator.evaluate(predictionsGBT)

println(s"RMSE Random Forest: $rmseRF")
println(s"RMSE GBT: $rmseGBT")


RMSE Random Forest: 895.9559161395179
RMSE GBT: 867.5099355083967


evaluator = RegressionEvaluator: uid=regEval_1c49ce59d7e3, metricName=rmse, throughOrigin=false
rmseRF = 895.9559161395179
rmseGBT = 867.5099355083967


867.5099355083967

In [25]:
//Choix du meilleur modèle
val bestModel = if (rmseRF < rmseGBT) {
  println("➡️ Le meilleur modèle est Random Forest")
  modelRF
} else {
  println("➡️ Le meilleur modèle est GBT")
  modelGBT
}

// Test final sur de nouvelles données
val finalPredictions = bestModel.transform(testData)
finalPredictions.select("trip_duration", "prediction", "pickup_latitude", "pickup_longitude").show(10)


➡️ Le meilleur modèle est GBT
+-------------+------------------+------------------+------------------+
|trip_duration|        prediction|   pickup_latitude|  pickup_longitude|
+-------------+------------------+------------------+------------------+
|         1134| 978.3795778974707|40.788631439208984|-73.97711944580078|
|          592| 554.3954429566435|40.751060485839844| -73.9736099243164|
|         1677| 1247.921261791818| 40.76898956298828|-73.96556091308594|
|          303|291.55011892555086| 40.77553939819336|-73.95832824707031|
|          189| 365.6458398644942|40.733150482177734|-73.98651123046875|
|          233| 279.0284082563539|40.748809814453125|-73.97776794433594|
|          998| 945.1849443711409| 40.75951385498047|-73.97789001464844|
|           95| 328.1358937808762| 40.79323959350586|-73.96708679199219|
|         1041|1178.7157384035297|40.815181732177734|-73.95893859863281|
|          500| 601.8732965340262| 40.76447296142578|-73.96620178222656|
+-------------+------

bestModel = pipeline_e0920f277056
finalPredictions = [id: string, vendor_id: int ... 22 more fields]


[id: string, vendor_id: int ... 22 more fields]

**interprétations:**

**1.Sélection du meilleur modèle :**

- Après comparaison des RMSE entre Random Forest (≈896 s) et GBT (≈868 s), le GBT a été retenu comme le meilleur modèle, car il prédit la durée des courses avec une erreur légèrement plus faible.

- Cela montre que l’apprentissage itératif et pondéré du GBT capture mieux les variations dans les données que le Random Forest classique.

**2.Prédictions finales :**

Les prédictions du modèle GBT sur le jeu de test montrent que :

- Pour certaines courses longues (ex. 1677 s), le modèle sous-estime légèrement (prédit 1248 s).

- Pour des courses courtes (ex. 303 s), le modèle est assez précis (prédit 292 s).

- La plupart des prédictions sont proches des valeurs réelles, surtout pour les durées autour de la moyenne.

- Les colonnes pickup_latitude et pickup_longitude permettent éventuellement d’analyser la performance du modèle selon la zone géographique.

**3.Conclusion générale :**

- Le GBT est le modèle final recommandé pour prédire la durée des courses de taxi à New York sur ce jeu de données.

- Les prédictions sont raisonnablement précises pour la majorité des courses, mais certaines valeurs extrêmes restent difficiles à estimer avec exactitude.

- Le modèle peut être utilisé pour la planification, l’optimisation des trajets ou l’analyse de la demande par zone et heure.